In [3]:
import pandas as pd
import geopandas as gpd
import osmnx as ox

from DataAnalysis import DataPreparation as DataPreparation
from DataAnalysis import DataMethodPreparation as DataMethodPreparation
from AssastmentMethod import AspectLaw as AspectLaw
from AssastmentMethod import AspectPhysical as AspectPhysical
from AssastmentMethod import AspectSpatial as AspectSpatial
from AssastmentMethod import AspectEconomic as AspectEconomic
from AssastmentMethod import General as General

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Way51\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Way51\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
h:\Documents\Python\Risks_of_investing_in_cultural_heritage\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Загрузка исходного файла с ОКН
heritage_origin = pd.read_csv('Example_files/All_heritage.csv')

C:\Users\Way51\AppData\Local\Temp\ipykernel_25336\1288108848.py:2: DtypeWarning: Columns (1,50) have mixed types. Specify dtype option on import or set low_memory=False.
  heritage_origin = pd.read_csv('Example_files/All_heritage.csv')


In [3]:
# Выборка объектов в нужном регионе
REGION = 'Вологодская область'
ch_region, ch_region_coor, ch_region_none = DataPreparation.separator_reestr(heritage_origin, REGION) # Общий датасет, ГДФ(что можно найти), ДФ без координат оставшиеся

  0%|          | 0/975 [00:00<?, ?it/s]

100%|██████████| 975/975 [00:00<00:00, 4202.40it/s]


0    Вологодская область
1    Вологодская область
2    Вологодская область
Name: Регион, dtype: object
Всего объектов: 975
Количество объектов с координатами: 292
Количество объектов без координат: 683


In [4]:
# Загрузка отчета о кадастровой оценке
report = gpd.read_file('Example_files/buildings_report.geojson') 
report = DataPreparation.number_report(report)

h:\Documents\Python\Risks_of_investing_in_cultural_heritage\DataAnalysis.py:63: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '35-164339' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  gdf.loc[i, 'учетный номер'] = desc


In [71]:
# УДАЛИТЬ
ch_region['Кадастровый номер'] = ''
for i in range(len(ch_region)):
    for j in range(len(report)):
        if str(ch_region.loc[i, 'Номер в реестре']) == str(report.loc[j, 'Номер в реестре']) or str(ch_region.loc[i, 'учетный номер']) == str(report.loc[j, 'учетный номер']):
            ch_region.loc[i, 'Кадастровый номер'] = str(ch_region.loc[i, 'Кадастровый номер']) + str(report.loc[j, 'Кадастровый номер'])

In [5]:
# Выборка необходимых столбцов для анализа
ch_region = ch_region[['Номер в реестре', 'учетный номер', 'Объект', 'Регион', 'Полный адрес', 'Категория историко-культурного значения', 'Вид объекта', 'описание предмета охраны', 'Принадлежность к Юнеско', 'наименование документа', 'building_year', 'lat', 'lon', 'geometry']]
report = report[['Номер в реестре','учетный номер', 'Кадастровый номер', 'Кадастровые номера ЗУ, в пределах которых расположен данный ОН', 'Кадастровые номера помещений в объекте недвижимости', 'Площадь, кв.м', 'Назначение здания', 'Год завершения строительства', 'Количество этажей (в том числе подземных)', 'Количество подземных этажей', 'Материал наружных стен', 'Класс конструктивной системы', 'Группа капитальности', 'Объект признан аварийным ', 'Функциональная группа', 'Подгруппа', 'usage', 'area', 'cad_cost', 'geometry']]

# Преобразование столбцов в нужный формат
ch_region['Номер в реестре'] = ch_region['Номер в реестре'].astype(str)
ch_region['учетный номер'] = ch_region['учетный номер'].astype(str)

In [6]:
# Разделение в зависимости от наличие информации
report_1 = report[report['Номер в реестре'] != 0]
report_2 = report[report['Номер в реестре'] == 0]

# объединяем и удалаяем лишние стобцы
gdf_merged_1 = pd.merge(report_1, ch_region, on='Номер в реестре', how='inner')
gdf_merged_2 = pd.merge(report_2, ch_region, on='учетный номер', how='inner')

gdf_merged = pd.concat([gdf_merged_1, gdf_merged_2], ignore_index=True)
gdf_merged = gdf_merged.drop(['Номер в реестре_x', 'Номер в реестре_y', 'учетный номер_y', 'учетный номер_x'], axis = 1)

In [7]:
for i in range(len(gdf_merged)):
    if gdf_merged.loc[i, 'geometry_y'] != None:
        gdf_merged.loc[i, 'geometry_y'] = gdf_merged.loc[i, 'geometry_y'].centroid # Если геометрия не пустая, то из полигона делаем точку
    else:
        gdf_merged.loc[i, 'geometry_y'] = gdf_merged.loc[i, 'geometry_x']
gdf_merged.rename(columns={'geometry_y':'geometry'}, inplace = True)
gdf_merged = gpd.GeoDataFrame(gdf_merged,geometry='geometry', crs='EPSG:4326')

In [8]:
gdf_merged['geometry']=gdf_merged['geometry'].centroid

C:\Users\Way51\AppData\Local\Temp\ipykernel_25336\1312695513.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_merged['geometry']=gdf_merged['geometry'].centroid


In [9]:
# готовим ДФ для последующего анализа
gdf_merged = DataMethodPreparation.start_all(gdf_merged)

# добавляекм кадастровую стоимость
cad_cost = pd.read_excel('Example_files/report_cost_1.xlsx')
cad_cost = cad_cost[['Кадастровый номер','Аварийность объекта','Экономическое устаревание', 'Эффективный возраст', 'Срок экономической жизни, лет', 'Физический износ', 'Совокупный износ','Удельный показатель кадаcтровой стоимости объекта недвижимости, руб./кв.м', 'Кадастровая стоимость (при оценке), руб']]
gdf_merged = pd.merge(gdf_merged, cad_cost, how='left', on='Кадастровый номер')
gdf_merged = gdf_merged.drop('geometry_x', axis=1)

In [13]:
# перенести в отдельный модуль
def parcer_osm(region, tag_str, tag_def = True): #На первом месте указываем тег, на втором его значение по умолчанию выбираются все значения
    tags = {tag_str: tag_def}
    objects = ox.features.features_from_polygon(region.unary_union, tags)
    return objects

def data_osm(city):
    region = ox.geocode_to_gdf(query=city).reset_index()
    building_osm = parcer_osm(region,'building').reset_index()
    building_osm = building_osm[building_osm['element_type'] != 'node'].reset_index()
    try:
        shop = parcer_osm(region,'shop').reset_index()
    except:
        print('Не найдены магазины')
    try:
        cafe = parcer_osm(region,'amenity', ['bar', 'cafe', 'fast_food', 'food_court', 'pub', 'restaurant']).reset_index()
    except:
        print('Не найдены точки питания')
    try:
        public_transport = parcer_osm(region,'public_transport').reset_index()
    except:
        print('Не найдены остановки общественного транспорта')
    return(region, building_osm, shop, cafe, public_transport)

In [14]:
# Скачиваем данные из OSM по двум исследуемым городам
region, building_osm, shop, cafe, public_transport = data_osm('город Вологда')
#region_t, building_osm_t, shop_t, cafe_t, public_transport_t = data_osm('город Тотьма')

In [15]:
# Выбираем объекты расположенные в городе Вологда
heritage_vologda = gpd.overlay(gdf_merged, region, how='intersection')
#heritage_totma = gpd.overlay(gdf_merged, region_t, how='intersection')

In [16]:
# Дополнительная информация из викимапии
wiki_gdf = pd.read_csv('Example_files/file_from_wiki_vologda.csv')
wiki_gdf = gpd.GeoDataFrame(wiki_gdf, geometry=gpd.points_from_xy(wiki_gdf.lon, wiki_gdf.lat), crs='EPSG:4326')

wiki_gdf = wiki_gdf.to_crs('EPSG:32637')
heritage_vologda = heritage_vologda.to_crs('EPSG:32637')

heritage_vologda['geometry'] = heritage_vologda['geometry'].buffer(20)
heritage_vologda['arch_style']=''

for i in range(len(heritage_vologda)):
        # Выбор точек внутри полигона
    points_within_polygon = wiki_gdf[wiki_gdf.geometry.within(heritage_vologda.geometry[i])]
    # Группировка значений
    grouped_values = points_within_polygon.groupby('Unnamed: 0')['arch_style'].apply(list).reset_index()
    if grouped_values.empty != True:
        heritage_vologda.loc[i, 'arch_style'] = grouped_values['arch_style'][0][0]
    grouped_values['arch_style'] = ''

for i in range(len(building_osm)):
        # Выбор точек внутри полигона
    points_within_polygon = wiki_gdf[wiki_gdf.geometry.within(building_osm.geometry[i])]
    # Группировка значений
    grouped_values = points_within_polygon.groupby('Unnamed: 0')['arch_style'].apply(list).reset_index()
    if grouped_values.empty != True:
        building_osm.loc[i, 'arch_style'] = grouped_values['arch_style'][0][0]
    grouped_values['arch_style'] = ''




In [17]:
heritage_vologda = AspectPhysical.start_all(heritage_vologda)
heritage_vologda = AspectLaw.start_all(heritage_vologda)
heritage_vologda = AspectEconomic.start_all(heritage_vologda)

KeyError: 'actual_program'

In [18]:
heritage_vologda = AspectSpatial.start_all(heritage_vologda, shop, cafe, public_transport,building_osm)

2025-01-10 19:14:23.184 | INFO     | iduedu.modules.drive_walk_builder:get_walk_graph:216 - Downloading walk graph from OSM, it may take a while for large territory ...
Calculating the weights of the walk graph: 100%|██████████| 56454/56454 [00:02<00:00, 21054.59it/s]


UnboundLocalError: cannot access local variable 'building_osm' where it is not associated with a value

In [100]:
heritage_vologda = heritage_vologda.to_crs('EPSG:32637')
building_osm = building_osm.to_crs('EPSG:32637')

In [113]:
# DELETE THIS
def merge_reestr_building(gdf, building_osm):
    for i in range(len(building_osm)):
        poly = building_osm.loc[i,'geometry'].buffer(15)
        for j in range(len(gdf)):
            point = gdf.loc[j,'geometry']
            if poly.contains(point):
                building_osm.loc[i,'_building_year'] = gdf.loc[j,'building_year']
                building_osm.loc[i,'_arch_style'] = gdf.loc[j,'arch_style']
                building_osm.loc[i,'_material'] = gdf.loc[j,'material']
                building_osm.loc[i,'_building_floor'] = gdf.loc[j,'Количество этажей (в том числе подземных)']
                building_osm.loc[i,'_culture_heritage'] = 1
    return(building_osm)
                

In [129]:
# DELETE
def view_selection(gdf_vision, building_osm, gdf):
    gdf['score_historicity'] = 0
    for i in range(len(gdf_vision)):
        selection_buildings = gpd.overlay(building_osm, gdf_vision[gdf_vision.index == i], how='intersection')
        if selection_buildings['_culture_heritage'].count() >= 1:
            gdf.loc[i, 'score_historicity'] += 0.1
        floor_mean = selection_buildings['_building_floor'].fillna(1).astype(int).mean()
        if float(gdf.loc[i, 'Количество этажей (в том числе подземных)']) > float(floor_mean - 1) and float(gdf.loc[i, 'Количество этажей (в том числе подземных)']) < float(floor_mean + 1):
            gdf.loc[i, 'score_historicity'] += 0.3
        year_mean = selection_buildings['_building_year'].fillna(1917).astype(int).mean()
        if gdf.loc[i, 'building_year'] > (year_mean - 20) and gdf.loc[i, 'building_year'] < (year_mean + 20):
            gdf.loc[i, 'score_historicity'] += 0.3
    return(gdf)

In [134]:
heritage_vologda['total_score'] = 0
heritage_vologda['total_score'] = heritage_vologda['total_economic_score'] + heritage_vologda['total_law_score'] + heritage_vologda['total_physical_score']

In [72]:
# DELETE THIS

import json
# Закружаем критерии оценки
with open("Example_files/risks.json", encoding="UTF-8") as file_in:
    risks = json.load(file_in)

def risks_assastment(gdf):
    gdf['risks']=''
    for j in range(len(gdf)):
        for aspect in ['physical_aspect','law_aspect', 'economic_aspect']:
            for risk in risks[0][aspect]:
                if float(gdf.loc[j, risks[0][aspect][risk]['indicator']]) <= float(risks[0][aspect][risk]['value']):
                    gdf.loc[j, 'risks'] += str(risks[0][aspect][risk]['id']) + ';'
        gdf.loc[j, 'risks_num'] = len(gdf.loc[j, 'risks'].split(';')) - 1
    return(gdf)

In [20]:
heritage_vologda = General.calculate_scores(heritage_vologda)

KeyError: 'total_spatial_score'

In [21]:
heritage_vologda = General.risks_assastment(heritage_vologda)

In [22]:
heritage_vologda['geometry'] = heritage_vologda['geometry'].centroid

In [23]:
heritage_vologda.head(5)

,Номер в реестре,Кадастровый номер,"Кадастровые номера ЗУ, в пределах которых расположен данный ОН",Кадастровые номера помещений в объекте недвижимости,"Площадь, кв.м",Назначение здания,Год завершения строительства,Количество этажей (в том числе подземных),Количество подземных этажей,Материал наружных стен,...,total_physical_percent,score_protection,score_land_use,score_cadastral_integrity,total_law_score,total_law_percent,score_cadstral_value_building,score_cadstral_value_land,risks,risks_num
0,351811311630006,35:24:0201017:1222,35:24:0201017:67,None,233.5,Нежилое,1882.0,2,None,Рубленые,...,27.81,0.25,1.0,1.0,2.25,75.00,1.00,0.75,10001;10002;10004;30001;30002;,5.0
1,351510309010006,35:24:0202012:234,35:24:0202012:10,None,784.4,Нежилое,NaN,2,None,Рубленые,...,61.20,0.75,0.0,1.0,1.75,58.33,0.25,1.00,10001;10002;10004;,3.0
2,351711288060005,35:24:0304011:212,35:24:0304011:1075,35:24:0304011:707; 35:24:0304011:708,267.2,Нежилое,1906.0,2,None,Рубленые,...,38.53,1.00,0.0,1.0,2.00,66.67,1.00,1.00,10001;10002;10004;30002;,4.0
3,381310014200006,35:24:0201006:186,None,None,312.5,Жилое,1892.0,2,нет,Рубленые,...,38.53,0.25,0.0,1.0,1.25,41.67,1.00,0.25,10001;10002;10004;30001;,4.0
4,381310004990005,35:24:0202001:85,35:24:0202001:288,None,2340.7,Нежилое,NaN,5,1,Кирпичные,...,75.48,1.00,0.0,1.0,2.00,66.67,0.50,0.50,10001;10002;30002;,3.0


In [61]:
# Сохранение файла
heritage_vologda.to_file('output_files/heritage_vologda_ready.geojson')

In [5]:
from tobler.util import h3fy

# create hexagones for future analysis hex_area = 9.96 he
hex_vologda = h3fy(region, resolution=9, clip=True, buffer=False, return_geoms=True).reset_index()

hex_vologda = hex_vologda.to_crs('EPSG:32637')
hex_vologda['area'] = hex_vologda['geometry'].area/10000

hex_vologda.explore()

In [4]:
cafe = gpd.read_file('output_files/cafe.geojson')
region = gpd.read_file('output_files/region.geojson')

In [17]:
shop = gpd.read_file('output_files/shop.geojson')
public_transport = gpd.read_file('output_files/public_transport.geojson')

In [19]:
public_transport = public_transport.to_crs('EPSG:32637')

In [11]:
hex_vologda.crs

<Projected CRS: EPSG:32637>
Name: WGS 84 / UTM zone 37N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Between 36°E and 42°E, northern hemisphere between equator and 84°N, onshore and offshore. Djibouti. Egypt. Eritrea. Ethiopia. Georgia. Iraq. Jordan. Kenya. Lebanon. Russian Federation. Saudi Arabia. Somalia. Sudan. Syria. Türkiye (Turkey). Ukraine.
- bounds: (36.0, 0.0, 42.0, 84.0)
Coordinate Operation:
- name: UTM zone 37N
- method: Transverse Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [32]:
cafe.crs


<Projected CRS: EPSG:32637>
Name: WGS 84 / UTM zone 37N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Between 36°E and 42°E, northern hemisphere between equator and 84°N, onshore and offshore. Djibouti. Egypt. Eritrea. Ethiopia. Georgia. Iraq. Jordan. Kenya. Lebanon. Russian Federation. Saudi Arabia. Somalia. Sudan. Syria. Türkiye (Turkey). Ukraine.
- bounds: (36.0, 0.0, 42.0, 84.0)
Coordinate Operation:
- name: UTM zone 37N
- method: Transverse Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [29]:
def merge_hex_services(hex, cafe, shop, public_transport):
    for i in range(len(hex)):
            # Выбор точек внутри полигона
        points_within_polygon = cafe[cafe.geometry.within(hex.geometry[i])]
        # Группировка значений
        grouped_values = points_within_polygon.groupby('osmid')['amenity'].count().reset_index()
        hex.loc[i, 'cafe'] = grouped_values['amenity'].count()
        grouped_values['cafe'] = 0

        points_within_polygon = shop[shop.geometry.within(hex.geometry[i])]
        # Группировка значений
        grouped_values = points_within_polygon.groupby('osmid')['amenity'].count().reset_index()
        hex.loc[i, 'shop'] = grouped_values['amenity'].count()
        grouped_values['shop'] = 0

        # Выбор точек внутри полигона
        points_within_polygon = public_transport[public_transport.geometry.within(hex.geometry[i])]
        # Группировка значений
        grouped_values = points_within_polygon.groupby('osmid')['amenity'].count().reset_index()
        hex.loc[i, 'public_transport'] = grouped_values['amenity'].count()
        grouped_values['public_transport'] = 0
    return(hex)

In [26]:
for i in range(len(hex_vologda)):
            # Выбор точек внутри полигона
        points_within_polygon = cafe[cafe.geometry.within(hex_vologda.geometry[i])]
        # Группировка значений
        grouped_values = points_within_polygon.groupby('osmid')['amenity'].count().reset_index()
        hex_vologda.loc[i, 'cafe'] = grouped_values['amenity'].count()
        grouped_values['cafe'] = 0

In [35]:
hex_vologda = merge_hex_services(hex_vologda, cafe, shop, public_transport)

In [33]:
hex_vologda['cafe'] = 0

In [36]:
hex_vologda[hex_vologda['cafe'] !=0].explore('cafe')